In [7]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire, milling, conversions, structures

from fibsem.structures import BeamType, Point, FibsemRectangle


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
microscope, settings = utils.setup_session(config_path=r"C:\Users\Admin\Github\autolamella\autolamella")

2023-03-10 12:16:57,381 — root — INFO — connect_to_microscope:366 — Microscope client connecting to [10.0.0.1:7520]
Client connecting to [10.0.0.1:7520]...
Client connected to [10.0.0.1:7520]
2023-03-10 12:16:57,387 — root — INFO — connect_to_microscope:368 — Microscope client connected to [10.0.0.1:7520]
2023-03-10 12:16:57,388 — root — INFO — setup_session:227 — Finished setup for session: demo_2023-03-10-12-16-57PM


In [4]:

settings.image.beam_type = BeamType.ION
ref_image = acquire.new_image(microscope, settings.image)


2023-03-10 12:17:55,544 — root — INFO — autocontrast:473 — Running autocontrast on ION.
2023-03-10 12:17:56,151 — root — INFO — acquire_image:412 — acquiring new ION image.


In [35]:




pixelsize = ref_image.metadata.pixel_size.x
fiducial_centre = Point(-50e-6, -10e-6)
fiducial_length = 30e-6 # microscope_settings.protocol["fiducial"]["length"] 
fiducial_area = calculate_fiducial_area(settings, fiducial_centre, fiducial_length, pixelsize)

print(fiducial_area)

settings.image.reduced_area = fiducial_area
image = acquire.new_image(microscope, settings.image)
print(image.metadata.image_settings.reduced_area)


RESOLUTION:  [1536, 1024]
CENTRE: Point(x=-5e-05, y=-1e-05) Point(x=-512, y=-102)
LENGTH: 3e-05 460.5
LEFT:  0.02 TOP: 0.18
WIDTH: 0.30 HEIGHT: 0.45
FibsemRectangle(left=0.016764322916666685, top=0.175537109375, width=0.2998046875, height=0.44970703125)
2023-03-10 12:37:50,579 — root — INFO — autocontrast:473 — Running autocontrast on ION.
2023-03-10 12:37:51,246 — root — INFO — acquire_image:412 — acquiring new ION image.
FibsemRectangle(left=0.016764322916666685, top=0.175537109375, width=0.2998046875, height=0.44970703125)


## Statistics

In [3]:

import pandas as pd
import numpy as np


import plotly.express as px
from autolamella.structures import Experiment, Lamella
from autolamella.tools.stats import create_history_dataframe

In [4]:
path = "/home/patrick/github/autolamella/autolamella/log/test/test.yaml"

experiment = Experiment.load(path) 
df = create_history_dataframe(experiment)

In [8]:
display(df)|

SyntaxError: invalid syntax (2121213434.py, line 1)

In [11]:


fig = px.bar(df, x="stage", y="duration", color="petname", barmode="group")
fig.show()